### Import Libs

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import ast

### Df Params

In [42]:
get_email = False
af_by_name = False

### Getting df from db

In [43]:
columns = [f'Mise {i}' for i in range(1,6)] + [i for i in range(1,58)]
query_columns = 'Mises, AF'
if get_email:
    columns = ['Email'] + columns
    query_columns = query_columns + ', email'

df = pd.DataFrame(columns = columns)
AF_dict = pd.read_pickle('Dicionario.pkl')

conn = sqlite3.connect("banco_de_dados.db")
cursor = conn.cursor()
cmd = f"SELECT {query_columns} from questionnaire_s8 ORDER BY ID ASC"
cursor.execute(cmd)
query_result = cursor.fetchall()

for i in range(len(query_result)):
    disciplinas = ast.literal_eval(query_result[i][1])
    if af_by_name: disciplinas = list(map(lambda x: AF_dict[x],disciplinas))
    
    if len(disciplinas) < 57:
        missing = 57 - len(disciplinas)
        disciplinas += ['-' for i in range(missing)]
    
    if get_email:
        row = pd.Series([query_result[i][2]] + ast.literal_eval(query_result[i][0]) +  disciplinas, index=df.columns)
    else:
        row = pd.Series(ast.literal_eval(query_result[i][0]) +  disciplinas, index=df.columns)

    df = df.append(row, ignore_index=True)

In [44]:
df

,Mise 1,Mise 2,Mise 3,Mise 4,Mise 5,1,2,3,4,5,...,48,49,50,51,52,53,54,55,56,57
0,30.0,27.0,20.0,13.0,10.0,D-9,C-9,A-11,B-12,B-10,...,F-10,F-6,A-8,E-6,C-5,C-6,A-7,F-11,A-1,B-1
1,34.0,31.0,25.0,7.0,3.0,C-9,F-1,B-12,F-7,A-10,...,C-5,E-6,A-6,D-12,E-10,E-7,D-11,A-5,F-5,B-1
2,24.0,24.0,24.0,14.0,14.0,F-7,B-12,C-9,A-12,D-7,...,F-11,E-1,C-5,C-6,B-1,D-1,F-3,C-3,A-1,E-10
3,30.2,26.0,22.0,21.7,0.1,F-11,B-1,D-3,A-11,D-1,...,-,-,-,-,-,-,-,-,-,-
4,25.0,23.0,20.0,17.0,15.0,C-9,B-12,D-7,E-5,A-12,...,-,-,-,-,-,-,-,-,-,-
5,30.0,26.0,26.0,10.0,8.0,E-7,B-12,C-9,B-10,C-7,...,E-6,A-5,C-5,F-3,F-6,C-2,C-1,A-1,F-11,B-1
6,41.0,35.0,11.0,10.0,3.0,C-9,F-1,D-3,A-11,E-1,...,E-6,A-1,C-6,F-4,A-7,B-1,D-12,F-5,F-8,D-11
7,37.5,33.5,22.5,5.5,1.0,C-9,B-12,F-1,D-9,A-12,...,A-7,A-1,A-8,D-4,A-11,C-7,D-12,E-10,F-11,A-10
8,41.0,31.0,21.0,4.0,3.0,F-1,D-1,E-1,C-9,F-11,...,F-6,D-2,B-8,A-10,F-9,A-5,D-7,E-4,A-2,D-4
9,30.5,25.5,20.5,12.5,11.0,A-10,C-9,B-12,F-7,B-10,...,C-1,E-1,F-8,C-5,F-2,E-6,F-11,F-6,A-1,B-1
